### Classification based on extracted features

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

#from keras.applications.vgg16 import VGG16 as BaseModel
#from keras.applications.vgg16 import preprocess_input

from keras.applications.xception import Xception as BaseModel
from keras.applications.xception import preprocess_input

from keras.preprocessing import image
from keras.models import Model
import numpy as np

In [4]:
model = BaseModel(weights='imagenet', include_top=True)

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [8]:
features_model = Model(input=model.input, output=model.layers[-2].output)

/raid/data2/nshvai/classification/env/.classif3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  """Entry point for launching an IPython kernel.


In [9]:
features_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [27]:
test_dir = 'data/training_folder/test'
val_dir = 'data/training_folder/val'
train_dir = 'data/training_folder/train'
image_size = tuple(model.input.shape[1:3].as_list()) ###(224,224)

In [28]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

test_filenames = test_generator.filenames
val_filenames = val_generator.filenames
train_filenames = train_generator.filenames

test_nb_samples = len(test_filenames)
val_nb_samples = len(val_filenames)
train_nb_samples = len(train_filenames)

predict_test = features_model.predict_generator(test_generator,steps=test_nb_samples)
predict_val = features_model.predict_generator(val_generator,steps=val_nb_samples)
predict_train = features_model.predict_generator(train_generator,steps=train_nb_samples)

Found 341 images belonging to 17 classes.
Found 340 images belonging to 17 classes.
Found 681 images belonging to 17 classes.


In [29]:
predict_train.shape

(681, 2048)

Now we'll use classical ML algorithm on extracted features

Let us use only the first two classes

In [13]:
features_test = predict_test[test_generator.classes <=1]
features_train = predict_val[val_generator.classes <=1]

y_test = test_generator.classes[test_generator.classes <=1]
y_train = val_generator.classes[val_generator.classes <=1]

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
clf = LogisticRegression()

In [16]:
clf.fit(features_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
pred = clf.predict(features_test)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.925

For multiple classes

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [54]:
n_neighbors = 20

In [55]:
#clf = KNeighborsClassifier(n_neighbors=n_neighbors)
#clf = RandomForestClassifier(random_state=42)
clf = MLPClassifier()

In [56]:
features_test = predict_test
features_train = predict_train

y_test = test_generator.classes
y_train = train_generator.classes

In [57]:
clf.fit(features_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [58]:
pred = clf.predict(features_test)

In [59]:
accuracy_score(pred, y_test)

0.9120234604105572